In [ ]:
import tomllib
from pathlib import Path

import geopandas as gpd
import pandas as pd
import polars as pl

# For PBA 2050: see p77 in
# https://www.planbayarea.org/sites/default/files/documents/Plan_Bay_Area_2050_Forecasting_Modeling_Report_October_2021.pdf

In [ ]:
with open("configs/2023.toml", "rb") as f:
    config = tomllib.load(f)

express_lanes_dir = Path(
    r"Y:\networks\projects\@regional_background\MTC_Committed_Exp_Lanes"
)
convert_hov2hot_filepath = express_lanes_dir / "convert_hov2hot" / "mod.dat"

In [ ]:
tolled_categories = [
    "TOLLAM_DA",
    "TOLLAM_SR2",
    "TOLLMD_DA",
    "TOLLMD_SR2",
    "TOLLPM_DA",
    "TOLLPM_SR2",
    "TOLLEV_DA",
    "TOLLEV_SR2",
]

cpi1989 = 126.4
cpi2000 = 180.2
cpi2010 = 227.47

In [ ]:
network = gpd.read_file(
    Path(config["champ"]["forecast"]["model_run_dir"]) / "FREEFLOW.shp"
)
# network = pl.from_pandas(network.drop(columns="geometry"))

In [ ]:
convert_hov2hot_df = pd.read_csv(convert_hov2hot_filepath)

In [ ]:
convert_hov2hot_df = network.drop(columns=tolled_categories).merge(
    convert_hov2hot_df, on=["A", "B"], how="right"
)

In [ ]:
convert_hov2hot_df.plot(column="USE_y", categorical=True, legend=True)

In [ ]:
sr237wb = convert_hov2hot_df[convert_hov2hot_df["STREETNAME"] == 'SR 237 WB']
sr237wb[tolled_categories].describe()

In [ ]:
sr237wb[tolled_categories].div(sr237wb["DISTANCE"], axis=0).describe()

In [ ]:
print("SR-37 Westbound Per-LINK-SEGMENT Roadway Toll Cost")
print("in 2000 dollars:")
print(sr237wb[["TOLLAM_DA", "TOLLAM_SR2"]].mean() / cpi1989 * cpi2000)
print("in 2010 dollars:")
print(sr237wb[["TOLLAM_DA", "TOLLAM_SR2"]].mean() / cpi1989 * cpi2010)

In [ ]:
print("SR-37 Westbound Per-MILE Roadway Toll Cost")
print("in 2000 dollars:")
print(sr237wb[["TOLLAM_DA", "TOLLAM_SR2"]].div(sr237wb["DISTANCE"], axis=0).mean() / cpi1989 * cpi2000)
print("in 2010 dollars:")
print(sr237wb[["TOLLAM_DA", "TOLLAM_SR2"]].div(sr237wb["DISTANCE"], axis=0).mean() / cpi1989 * cpi2010)
# these numbers are close enough to the MTC unmbers that I think the
# discrepancies are just from slightly differently coded link lengths

In [ ]:
# the areas where use == 4 corresponds to the Dynamic Tolling portions on MTC's map
dynamic_tolling = convert_hov2hot_df[(convert_hov2hot_df["USE_y"] == 4) & (convert_hov2hot_df["STREETNAME"] != 'SR 237 WB')]
dynamic_tolling[tolled_categories].div(dynamic_tolling["DISTANCE"], axis=0).describe()

In [ ]:
print("dynamic tolling max (for both DA and SR2) = 100 in 1989 dollars")
print("max in 2000 dollars:", 1 / cpi1989 * cpi2000)
print("max in 2010 dollars:", 1 / cpi1989 * cpi2010)
print()
print("dynamic tolling min (DA) = 2.031250 in 1989 dollars")  # ignoring the weird one out for EV
print("min (DA) in 2000 dollars:", 2.031250 / cpi1989 * cpi2000)
print("min (DA) in 2010 dollars:", 2.031250 / cpi1989 * cpi2010)
print()
print("dynamic tolling min (SR2) = 0.375940 in 1989 dollars")  # ignoring the weird one out for EV
print("min (DA) in 2000 dollars:", 0.375940 / cpi1989 * cpi2000)
print("min (DA) in 2010 dollars:", 0.375940 / cpi1989 * cpi2010)

In [ ]:
# the areas where use == 1 corresponds to the Strategy T5 portions on MTC's map
# there's NO toll implemented on these
convert_hov2hot_df[(convert_hov2hot_df["USE_y"] == 1)][
    tolled_categories
].describe()